In [9]:
import pandas as pd
import os
import sys
print(sys.executable)
from tqdm import tqdm
import shutil
import librosa as lb



c:\Users\gioel\AppData\Local\Programs\Python\Python310\python.exe


In [14]:
# Imposta il percorso del file Excel
excel_path = "C:/Users/gioel/Desktop/noisereduced-uaspeech-control/data.xlsx"   

# Cartella di destinazione del dataset
output_dir = "C:/Users\gioel/Desktop/uaspeech_best_snr/audio/control"

# Lettura del file Excel
df = pd.read_excel(excel_path)

In [15]:
# Verifica colonne richieste
required_cols = {"ID", "BLOCK", "WORD", "W_CODE", "CHANNEL", "FILE_PATH", "SNR"}
if not required_cols.issubset(df.columns):
    raise ValueError(f"Colonne mancanti nel file Excel. Devono esserci: {required_cols}")

# Seleziona per ogni ID+WORD la riga con SNR massimo
best_rows = df.loc[df.groupby(["ID", "BLOCK", "WORD"])["SNR"].idxmax()].reset_index(drop=True)

# 📁 Creazione della cartella principale
os.makedirs(output_dir, exist_ok=True)

In [16]:
# Loop su ogni riga e copia del file
for _, row in best_rows.iterrows():
    person_id = row["ID"]
    block = row["BLOCK"]
    wcode = row["W_CODE"]
    file_path = row["FILE_PATH"]
    
    # Crea sottocartella per ID
    person_folder = os.path.join(output_dir, person_id)
    os.makedirs(person_folder, exist_ok=True)
    
    # Nome nuovo file
    if "UW" in str(wcode):
    # caso speciale UW: usa solo wcode
        new_name = f"{wcode}.wav"
    elif str(wcode).startswith(str(block)):
    # se wcode inizia già con block, non ripeterlo
        new_name = f"{wcode}.wav"
    else:
    # caso normale
        new_name = f"{block}_{wcode}.wav"

    new_path = os.path.join(person_folder, new_name)

    
    # Copia del file (solo se esiste)
    if os.path.exists(file_path):
        shutil.copy2(file_path, new_path)
    else:
        print(f"⚠️ File non trovato: {file_path}")

print("✅ Dataset creato con successo nella cartella 'control'")

✅ Dataset creato con successo nella cartella 'control'


In [18]:
# Carica il file originale
df = pd.read_excel("C:/Users/gioel/Desktop/noisereduced-uaspeech-control/data.xlsx")

# Mantieni solo le righe con SNR massimo per ogni parola
df_max_snr = df.loc[df.groupby(["ID", "BLOCK", "WORD"])["SNR"].idxmax()].reset_index(drop=True)
#  Salva il risultato in un nuovo file identico a data.xlsx ma filtrato
df_max_snr.to_excel("C:/Users/gioel/Desktop/uaspeech_best_snr/audio/control/data_control.xlsx", index=False)

print("✅ data_patient.xlsx creato con successo!")

✅ data_patient.xlsx creato con successo!
